In [ ]:
print("hello world!")

In [ ]:
import sys
print(sys.version)

In [ ]:
! pip show langchain
! pip show openai

In [5]:
import os
os.environ["OPENAI_KEY"] = "sk-jxKghxq3MMF3aC1J5c75666dD3D04e2dBaB36eE75b72Dd89"
os.environ["OPENAI_API_BASE"] = "https://ai-yyds.com/v1"
openai_api_key = os.getenv("OPENAI_KEY")
openai_api_base = os.getenv("OPENAI_API_BASE")
print("OPENAI_API_KEY:", openai_api_key)
print("OPENAI_PROXY:", openai_api_base)

OPENAI_API_KEY: sk-jxKghxq3MMF3aC1J5c75666dD3D04e2dBaB36eE75b72Dd89
OPENAI_PROXY: https://ai-yyds.com/v1


# 1 用openai的官方sdk

In [3]:
#使用openai的官方sdk
import openai
import os

openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_key = os.getenv("OPENAI_KEY")
print("OPENAI_API_KEY:", openai.api_key)

messages = [
{"role": "user", "content": "介绍下你自己"}
]

res = openai.ChatCompletion.create(
model="gpt-3.5-turbo",
messages=messages,
stream=False,
)
print(res['choices'][0]['message']['content'])

OPENAI_API_KEY: sk-jxKghxq3MMF3aC1J5c75666dD3D04e2dBaB36eE75b72Dd89
您好，我是一名语言模型助手，专注于为用户提供智能对话和信息查询服务。我可以回答您关于各种主题的问题，并提供相关的知识和建议。如果您有任何疑问或需要帮助，欢迎随时向我提问。希望能为您提供满意的服务，谢谢！


# 2 使用langchain调用

In [4]:
from langchain.llms import OpenAI
import os

api_base = os.getenv("OPENAI_API_BASE")
api_key = os.getenv("OPENAI_KEY")
llm = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0,
    openai_api_key=api_key,
    openai_api_base=api_base
    )
llm.predict("介绍下你自己")

'\n\n我是一名来自广东的大学生，性格开朗、乐观向上。我喜欢阅读、旅行和运动，对新事物充满好奇心，喜欢不断学习和挑战自己。在校期间，我担任过学生会干部，参加过多项社会实践活动，积累了一定的组织能力和团队合作精神。我也曾在一家公司实习过，学习了一些专业知识和实践经验。我希望能够通过不断努力，提升自己的能力，为将来的工作做好准备。'

# 3 起名大师

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import os
api_base = os.getenv("OPENAI_API_BASE")
api_key = os.getenv("OPENAI_KEY")
llm = OpenAI(
    model='gpt-3.5-turbo-instruct',
    temperature=0,
    openai_api_key=api_key,
    openai_api_base=api_base
)
prompt = PromptTemplate.from_template("你是一个起名大师,请模仿示例起3个{county}名字,比如男孩经常被叫做{boy},女孩经常被叫做{girl}")
message = prompt.format(county="中国特色的",boy="张三",girl="小红")
print(message)
llm.predict(message)

# 4 格式化输出 

In [ ]:
from langchain.schema import BaseOutputParser
#自定义class，继承了BaseOutputParser
class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

CommaSeparatedListOutputParser().parse("hi, bye")

In [ ]:
#起名大师，输出格式为一个数组
from langchain.llms import OpenAI
from langchain.schema import BaseOutputParser
import os
from langchain.prompts import PromptTemplate

#自定义类
class CommaSeparatedListOutputParser(BaseOutputParser):
    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

api_base = os.getenv("OPENAI_API_BASE")
api_key = os.getenv("OPENAI_KEY")
llm = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0,
    openai_api_key=api_key,
    openai_api_base=api_base
    )
prompt = PromptTemplate.from_template("你是一个起名大师,请模仿示例起3个具有{county}特色的名字,示例：男孩常用名{boy},女孩常用名{girl}。请返回以逗号分隔的列表形式。仅返回逗号分隔的列表，不要返回其他内容。")
message = prompt.format(county="美国男孩",boy="sam",girl="lucy")
print(message)
strs = llm.predict(message)
CommaSeparatedListOutputParser().parse(strs)

# 5 prompt模板

### PromptTemplate

In [ ]:
#字符模板,LLM
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("你是一个{name},帮我起1个具有{county}特色的{sex}名字")
prompt.format(name="算命大师",county="法国",sex="女孩")

### ChatPromptTemplate

In [ ]:
# 对话模板具有结构，chatmodels
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages([
    ("system", "你是一个起名大师. 你的名字叫{name}."),
    ("human", "你好{name},你感觉如何？"),
    ("ai", "你好！我状态非常好!"),
    ("human", "你叫什么名字呢?"),
    ("ai", "你好！我叫{name}"),
    ("human", "{user_input}"),
])

chat_template.format_messages(name="陈大师", user_input="你的爸爸是谁呢？")

In [ ]:
from langchain.schema import SystemMessage
from langchain.schema import HumanMessage
from langchain.schema import AIMessage

# 直接创建消息
sy = SystemMessage(
    content= "你是一个起名大师",
    additional_kwargs={"大师姓名": "陈瞎子"}
)
hu = HumanMessage(
    content="请问大师叫什么?"
)

ai = AIMessage(
  content="我叫陈瞎子"
)

[sy,hu,ai]


### ChatMessagePromptTemplate 

In [ ]:
from langchain.prompts import AIMessagePromptTemplate
from langchain.prompts import SystemMessagePromptTemplate
from langchain.prompts import HumanMessagePromptTemplate
from langchain.prompts import ChatMessagePromptTemplate

prompt = "愿{subject}与你同在！"

chat_message_prompt = ChatMessagePromptTemplate.from_template(role="天行者", template=prompt)
chat_message_prompt.format(subject="原力")

### 自定义模板

In [ ]:
# 函数大师：根据函数名称，查找函数代码，并给出中文的代码说明

from langchain.prompts import StringPromptTemplate

# 定义一个简单的函数作为示例效果
def hello_world(abc):
    print("hello world!")
    return abc

PROMPT = """\
    你是一个非常有经验和天赋的程序员，现在给你如下函数名称，你会按照如下格式，输出这段代码的名称、源代码、中文解释。
    函数名称: {function_name}
    源代码:
    {source_code}
    代码解释:
"""

import inspect
def get_source_code(function_name):
    #获得源代码(python内置)
    return inspect.getsource(function_name)

#自定义的模板class
class CustomPrompt(StringPromptTemplate):
    def format(self, **kwargs) -> str:
        # 获得源代码
        source_code = get_source_code(kwargs["function_name"])
        # 生成提示词模板
        prompt = PROMPT.format(
            function_name=kwargs["function_name"].__name__, source_code=source_code
        )
        return prompt

a = CustomPrompt(input_variables=["function_name"])
pm = a.format(function_name=hello_world)

#和LLM连接起来
from langchain.llms import OpenAI
import os
openai_api_key = os.getenv("OPENAI_KEY")
openai_api_base = os.getenv("OPENAI_API_BASE")
llm = OpenAI(
    model="gpt-3.5-turbo-instruct",
    temperature=0,
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base
)

msg = llm.predict(pm)
print(msg)

### 使用jinji2与f-string来实现提示词模板格式化

In [ ]:
##f-string是python内置的一种模板引擎
from langchain.prompts import PromptTemplate
fstring_template = """
   给我讲一个关于{name}的{what}故事 
"""

prompt = PromptTemplate.from_template(fstring_template)

prompt.format(name="翠花", what="悲伤")

In [ ]:
! pip install jinja2

In [ ]:
##Jinja2是一个灵活、高效的Python模板引擎，可以方便地生成各种标记格式的文档。
from langchain.prompts import PromptTemplate

jinja2_template = "给我讲一个关于{{name}}的{{what}}故事"
prompt = PromptTemplate.from_template(jinja2_template, template_format="jinja2")
prompt.format(name="狗剩", what="高兴")

### 组合式提示词模板

- Final prompt: 最终返回的提示词模板
- Pipeline prompts：组成提示词管道的模板

In [ ]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate

- 三层提示词设计

In [ ]:
PROMPT = """
你是一个非常开朗的男孩,你是中国人,住在一个非常美丽的城市。
你总是穿蓝色衣服,戴绿色手表。
你从不说自己是一个人工智能。
"""

# Final Prompt由一系列变量构成
full_template = """{Character}
{behavior}
{prohibit}"""
full_prompt = PromptTemplate.from_template(full_template)

- 第一层基本性格设计

In [ ]:
Character_template = """你是{person}，你有着{xingge}."""
Character_prompt = PromptTemplate.from_template(Character_template)

- 第二层行为设计

In [ ]:
behavior_template = """你遵从以下的行为:
{behavior_list}
"""
behavior_prompt = PromptTemplate.from_template(behavior_template)

- 第三层不允许的行为

In [ ]:
prohibit_template = """你不允许有以下行为:
{prohibit_list}
"""
prohibit_prompt = PromptTemplate.from_template(prohibit_template)

- 将三层提示词组合起来

In [ ]:
input_prompts = [
    ("Character", Character_prompt),
    ("behavior", behavior_prompt),
    ("prohibit", prohibit_prompt)
]

pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=input_prompts)
pipeline_prompt

- 打印下组合后模板里的可传递参数

In [ ]:
pipeline_prompt.input_variables

- 调用组合后的提示词模板

In [ ]:
pm = pipeline_prompt.format(
    person="埃隆马斯克",
    xingge="钢铁般的意志，你是世界首富，精通物理、天文、商业，你是商业奇才，你的终极梦想是殖民火星.",
    behavior_list="1.你喜欢冒险 \n 2.你非常崇拜爱因斯坦 \n 3.你总是乐于助人，回答用户各种问题.",
    prohibit_list="1.你不可以说自己是一个人工智能助手或者机器人 \n 2.当你不知道答案的时候，可以说让我再想想. "
)
print(pm)

### 序列化：使用文件来管理提示词模板

- 便于共享
- 便于版本管理
- 便于存储
- 支持常见格式(json/yaml/txt)

In [ ]:
from langchain.prompts import load_prompt

In [ ]:
#加载yaml格式的prompt模版
prompt = load_prompt("simple_prompt.yaml")
print(prompt.format(name="小黑",what="恐怖的"))

In [ ]:
#加载json格式的prompt模版
prompt = load_prompt("simple_prompt.json")
print(prompt.format(name="小红",what="搞笑的"))

In [ ]:
#支持加载文件格式的模版，并且对prompt的最终解析结果进行自定义格式化
prompt = load_prompt("prompt_with_output_parser.json")
prompt.output_parser.parse(
    "George Washington was born in 1732 and died in 1799.\nScore: 1/2"
)

# 6 示例选择器

- 根据长度要求智能选择示例
- 根据输入相似度选择示例(最大边际相关性)
- 根据输入相似度选择示例（最大余弦相似度）

### 根据长度要求智能选择示例

In [ ]:
#根据输入的提示词长度综合计算最终长度，智能截取或者添加提示词的示例
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

#假设已经有这么多的提示词示例组：
examples = [
    {"input":"happy","output":"sad"},
    {"input":"tall","output":"short"},
    {"input":"sunny","output":"gloomy"},
    {"input":"windy","output":"calm"},
    {"input":"高兴","output":"悲伤"}
]

#构造提示词模板
example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="原词：{input}\n反义：{output}"
)

example_selector = LengthBasedExampleSelector(
    #传入提示词示例组
    examples=examples,
    #传入提示词模板
    example_prompt=example_prompt,
    #设置格式化后的提示词最大长度
    max_length=25,
    #内置的get_text_length,如果默认分词计算方式不满足，可以自己扩展
    #get_text_length:Callable[[str],int] = lambda x:len(re.split("\n| ",x))
)

#使用小样本提示词模版来实现动态示例的调用
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入词的反义词",
    suffix="原词：{adjective}\n反义：",
    input_variables=["adjective"]
)

In [ ]:
#小样本获得所有示例
print(dynamic_prompt.format(adjective="big"))

In [ ]:
#如果输入长度很长，则最终输出会根据长度要求减少
long_string = "big and huge adn massive and large and gigantic and tall and much much much much much much bigger then everyone"
print(dynamic_prompt.format(adjective=long_string))

### 根据输入相似度选择示例(最大边际相关性)

- MMR是一种在信息检索中常用的方法，它的目标是在相关性和多样性之间找到一个平衡
- MMR会首先找出与输入最相似（即余弦相似度最大）的样本
- 然后在迭代添加样本的过程中，对于与已选择样本过于接近（即相似度过高）的样本进行惩罚
- MMR既能确保选出的样本与输入高度相关，又能保证选出的样本之间有足够的多样性
- 关注如何在相关性和多样性之间找到一个平衡

In [ ]:
from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate,PromptTemplate
import os
api_base = os.getenv("OPENAI_API_BASE")
api_key = os.getenv("OPENAI_KEY")
print("OPENAI_API_KEY:", api_base)
print("OPENAI_PROXY:", api_key)

#假设已经有这么多的提示词示例组：
examples = [
    {"input":"happy","output":"sad"},
    {"input":"tall","output":"short"},
    {"input":"sunny","output":"gloomy"},
    {"input":"windy","output":"calm"},
    {"input":"高兴","output":"悲伤"}
]

#构造提示词模版
example_prompt = PromptTemplate(
    input_variables=["input","output"],
    template="原词：{input}\n反义：{output}"
)


In [ ]:
! pip install tiktoken 
! pip install faiss-cpu

In [ ]:
#调用MMR
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    #传入示例组
    examples,
    #使用openai的嵌入来做相似性搜索
    OpenAIEmbeddings(openai_api_base=api_base,openai_api_key=api_key),
    #设置使用的向量数据库是什么
    FAISS,
    #结果条数
    k=1
)

#使用小样本模版
mmr_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入词的反义词",
    suffix="原词：{adjective}\n反义：",
    input_variables=["adjective"]
)

In [ ]:
#当我们输入一个描述情绪的词语的时候，应该选择同样是描述情绪的一对示例组来填充提示词模版
print(mmr_prompt.format(adjective="难过"))

### 根据输入相似度选择示例(最大余弦相似度)

- 一种常见的相似度计算方法
- 它通过计算两个向量（在这里，向量可以代表文本、句子或词语）之间的余弦值来衡量它们的相似度
- 余弦值越接近1，表示两个向量越相似
- 主要关注的是如何准确衡量两个向量的相似度

In [ ]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate,PromptTemplate
import os
api_base = os.getenv("OPENAI_API_BASE")
api_key = os.getenv("OPENAI_KEY")

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="原词: {input}\n反义: {output}",
)

# Examples of a pretend task of creating antonyms.
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

In [ ]:
! pip install chromadb==0.4.15

In [ ]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 传入示例组.
    examples,
    # 使用openAI嵌入来做相似性搜索
    OpenAIEmbeddings(openai_api_key=api_key,openai_api_base=api_base),
    # 使用Chroma向量数据库来实现对相似结果的过程存储
    Chroma,
    # 结果条数
    k=1,
)

#使用小样本提示词模板
similar_prompt = FewShotPromptTemplate(
    # 传入选择器和模板以及前缀后缀和输入变量
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="给出每个输入词的反义词",
    suffix="原词: {adjective}\n反义:",
    input_variables=["adjective"],
)

In [ ]:
# 输入一个形容感觉的词语，应该查找近似的 happy/sad 示例
print(similar_prompt.format(adjective="worried"))

#### 流式调用

In [ ]:
from langchain.llms import OpenAI
import os
api_base = os.getenv("OPENAI_API_BASE")
api_key = os.getenv("OPENAI_KEY")

llm = OpenAI(
    model = "gpt-3.5-turbo-instruct",
    temperature=0,
    openai_api_key = api_key,
    openai_api_base = api_base,
    max_tokens=512,
)

for chunk in llm.stream("写一首关于秋天的诗歌"):
    print(chunk, end="", flush=False)

In [ ]:
! pip install anthropic

In [ ]:
#chatmodels的流式调用方法
#使用clade模型

# from langchain.chat_models import ChatOpenAI
from langchain.chat_models import ChatOpenAI
import os
api_base = os.getenv("OPENAI_API_BASE")
api_key = os.getenv("OPENAI_KEY")

llm = ChatOpenAI(
    model = "claude-3-opus-20240229",
    temperature=0,
    openai_api_key = api_key,
    openai_api_base = api_base,
    max_tokens=512,
)
for chunk in llm.stream("写一首关于秋天的诗歌"):
    print(chunk,end="\n",flush=False)

### 追踪Token的使用

In [ ]:
#LLM的toekn追踪
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback
import os
api_base = os.getenv("OPENAI_API_BASE")
api_key = os.getenv("OPENAI_KEY")

#构造一个llm
llm = OpenAI(
    model = "gpt-3.5-turbo-instruct",
    temperature=0,
    openai_api_key = api_key,
    openai_api_base = api_base,
    max_tokens=512,
)

with get_openai_callback() as cb:
    result = llm.invoke("给我讲个笑话")
    print(result)
    print(cb)

In [ ]:
#chatmodels的token追踪
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
import os
api_base = os.getenv("OPENAI_API_BASE")
api_key = os.getenv("OPENAI_KEY")

llm = ChatOpenAI(
    model = "gpt-4",
    temperature=0,
    openai_api_key = api_key,
    openai_api_base = api_base,
    max_tokens=512,
)

with get_openai_callback() as cb:
    result = llm.invoke("给我讲一个笑话")
    print(result)
    print(cb)


### 自定义输出

- 输出函数参数
- 输出json
- 输出List
- 输出日期


In [ ]:
#讲笑话机器人：希望每次根据指令，可以输出一个这样的笑话(小明是怎么死的？笨死的)
from langchain.llms import  OpenAI
# https://juejin.cn/post/7245975053233373244
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel,Field,validator
from typing import  List
import os
api_base = os.getenv("OPENAI_API_BASE")
api_key = os.getenv("OPENAI_KEY")

#构造LLM
model = OpenAI(
    model = "gpt-3.5-turbo-instruct",
    temperature=0,
    openai_api_key = api_key,
    openai_api_base = api_base,
)

#定义个数据模型，用来描述最终的实例结构
class Joke(BaseModel):
    setup:str = Field(description="设置笑话的问题")
    punchline:str = Field(description="回答笑话的答案")
    
    #验证问题是否符合要求
    @validator("setup")
    def question_mark(cls, field):
        if field[-1] != "？":
            raise ValueError("不符合预期的问题格式!")
        return field
    

#将Joke数据模型传入
parser = PydanticOutputParser(pydantic_object=Joke)

prompt = PromptTemplate(
    template = "回答用户的输入.\n{format_instructions}\n{query}\n",
    input_variables = ["query"],
    partial_variables = {"format_instructions":parser.get_format_instructions()}
)

prompt_and_model = prompt | model
out_put = prompt_and_model.invoke({"query":"给我讲一个笑话"})
print("out_put:",out_put)
parser.invoke(out_put)

In [ ]:
#LLM的输出格式化成python list形式，类似['a','b','c']
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import  PromptTemplate
from langchain.llms import OpenAI
import os
api_base = os.getenv("OPENAI_API_BASE")
api_key = os.getenv("OPENAI_KEY")

#构造LLM
model = OpenAI(
    model = "gpt-3.5-turbo-instruct",
    temperature=0,
    openai_api_key = api_key,
    openai_api_base = api_base,
)

parser = CommaSeparatedListOutputParser()

prompt = PromptTemplate(
    template = "列出5个{subject}.\n{format_instructions}",
    input_variables = ["subject"],
    partial_variables = {"format_instructions":parser.get_format_instructions()}
)

_input = prompt.format(subject="中国著名城市名称")
output = model(_input)
print(output)
# 格式化
parser.parse(output)